<a href="https://colab.research.google.com/github/sujatasaini/KMNIST_Dataset_classification/blob/master/GRU_SVM_Dropout_Kuzushiji_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras

In [0]:
import requests

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x, total, unit: x  # If tqdm doesn't exist, replace it with a function that does nothing
    print('**** Could not import tqdm. Please install tqdm for download progressbars! (pip install tqdm) ****')

# Python2 compatibility
try:
    input = raw_input
except NameError:
    pass

download_dict = {
    '1) Kuzushiji-MNIST (10 classes, 28x28, 70k examples)': {
        '1) MNIST data format (ubyte.gz)':
            ['http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-images-idx3-ubyte.gz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-labels-idx1-ubyte.gz'],
        '2) NumPy data format (.npz)':
            ['http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz'],
    },
    '2) Kuzushiji-49 (49 classes, 28x28, 270k examples)': {
        '1) NumPy data format (.npz)':
            ['http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-labels.npz'],
    },
    '3) Kuzushiji-Kanji (3832 classes, 64x64, 140k examples)': {
        '1) Folders of images (.tar)':
            ['http://codh.rois.ac.jp/kmnist/dataset/kkanji/kkanji.tar'],
    }

}

# Download a list of files
def download_list(url_list):
    for url in url_list:
        path = url.split('/')[-1]
        r = requests.get(url, stream=True)
        with open(path, 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            print('Downloading {} - {:.1f} MB'.format(path, (total_length / 1024000)))

            for chunk in tqdm(r.iter_content(chunk_size=1024), total=int(total_length / 1024) + 1, unit="KB"):
                if chunk:
                    f.write(chunk)
    print('All dataset files downloaded!')

# Ask the user about which path to take down the dict
def traverse_dict(d):
    print('Please select a download option:')
    keys = sorted(d.keys())  # Print download options
    for key in keys:
        print(key)

    userinput = input('> ').strip()

    try:
        selection = int(userinput) - 1
    except ValueError:
        print('Your selection was not valid')
        traverse_dict(d)  # Try again if input was not valid
        return

    selected = keys[selection]

    next_level = d[selected]
    if isinstance(next_level, list):  # If we've hit a list of downloads, download that list
        download_list(next_level)
    else:
        traverse_dict(next_level)     # Otherwise, repeat with the next level

traverse_dict(download_dict)

Please select a download option:
1) Kuzushiji-MNIST (10 classes, 28x28, 70k examples)
2) Kuzushiji-49 (49 classes, 28x28, 270k examples)
3) Kuzushiji-Kanji (3832 classes, 64x64, 140k examples)
> 1
Please select a download option:
1) MNIST data format (ubyte.gz)
2) NumPy data format (.npz)
> 1


  0%|          | 14/17740 [00:00<02:34, 114.88KB/s]

100%|██████████| 29/29 [00:00<00:00, 223.72KB/s]


  0%|          | 14/2970 [00:00<00:24, 118.54KB/s]

 83%|████████▎ | 5/6 [00:00<00:00, 2324.23KB/s]

All dataset files downloaded!


In [0]:
"""Implementation of GRU+SVM model for MNIST with Dropout"""
import numpy as np
import tensorflow as tf
import time
from tensorflow.examples.tutorials.mnist import input_data

  
data = input_data.read_data_sets('kmnist/kmnist/dataset/kmnist/data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting kmnist/kmnist/dataset/kmnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting kmnist/kmnist/dataset/kmnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting kmnist/kmnist/dataset/kmnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting kmnist/kmnist/dataset/kmnist/data/t10k-labels-idx1-ubyt

In [0]:
# hyper-parameters
BATCH_SIZE = 256
CELL_SIZE = 256
DROPOUT_P_KEEP = 0.85
EPOCHS = 100
LEARNING_RATE = 1e-3
NUM_CLASSES = 10
SVM_C = 1

# dataset dimension
CHUNK_SIZE = 28
NUM_CHUNKS = 28

CHECKPOINT_PATH = 'checkpoint/'
MODEL_NAME = 'model.ckpt'

LOGS_PATH = 'logs/rnn/'

x = tf.placeholder(dtype=tf.float32, shape=[None, NUM_CHUNKS, CHUNK_SIZE], name='x_input')
y = tf.placeholder(dtype=tf.float32, shape=[None, NUM_CLASSES], name='y_input')
h = tf.placeholder(dtype=tf.float32, shape=[None, CELL_SIZE], name='state')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
p_keep = tf.placeholder(dtype=tf.float32, name='p_keep')


def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


def recurrent_neural_network(x):
    with tf.name_scope('weights_and_biases'):
        with tf.name_scope('weights'):
            xav_init = tf.contrib.layers.xavier_initializer
            weight = tf.get_variable('weights', shape=[CELL_SIZE, NUM_CLASSES], initializer=xav_init())
            variable_summaries(weight)
        with tf.name_scope('biases'):
            bias = tf.get_variable('biases', initializer=tf.constant(0.1, shape=[NUM_CLASSES]))
            variable_summaries(bias)

    cell = tf.contrib.rnn.GRUCell(CELL_SIZE)
    drop_cell = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=p_keep)
    
    outputs, states = tf.nn.dynamic_rnn(drop_cell, x, initial_state=h, dtype=tf.float32)

    states = tf.identity(states, name='H')
    hf = tf.transpose(outputs, [1, 0, 2])
    last = tf.gather(hf, int(hf.get_shape()[0]) - 1)
    with tf.name_scope('Wx_plus_b'):
        output = tf.matmul(last, weight) + bias
        tf.summary.histogram('pre-activations', output)

    return output, weight, states


def train_neural_network(x):
    prediction, weight, states = recurrent_neural_network(x)

    with tf.name_scope('loss'):
        regularization_loss = 0.5 * tf.reduce_sum(tf.square(weight))
        hinge_loss = tf.reduce_sum(tf.square(tf.maximum(tf.zeros([BATCH_SIZE, NUM_CLASSES]),
                                                        1 - tf.cast(y, tf.float32) * prediction)))
        with tf.name_scope('loss'):
            cost = regularization_loss + SVM_C * hinge_loss
    tf.summary.scalar('loss', cost)

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    with tf.name_scope('accuracy'):
        predicted_class = tf.sign(prediction)
        predicted_class = tf.identity(predicted_class, name='prediction')
        with tf.name_scope('correct_prediction'):
            correct = tf.equal(tf.argmax(predicted_class, 1), tf.argmax(y, 1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge_all()

    timestamp = str(time.asctime())
    writer = tf.summary.FileWriter(LOGS_PATH + timestamp, graph=tf.get_default_graph())

    saver = tf.train.Saver(max_to_keep=10)

    current_state = np.zeros([BATCH_SIZE, CELL_SIZE])

    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    with tf.Session() as sess:
        sess.run(init_op)

        checkpoint = tf.train.get_checkpoint_state(CHECKPOINT_PATH)

        if checkpoint and checkpoint.model_checkpoint_path:
            saver.restore(sess, tf.train.latest_checkpoint(CHECKPOINT_PATH))
        try:
            for epoch in range(EPOCHS):
                epoch_loss = 0
                for _ in range(int(data.train.num_examples / BATCH_SIZE)):
                    epoch_x, epoch_y = data.train.next_batch(BATCH_SIZE)
                    epoch_y[epoch_y == 0] = -1

                    epoch_x = epoch_x.reshape((BATCH_SIZE, NUM_CHUNKS, CHUNK_SIZE))

                    feed_dict = {x: epoch_x, y: epoch_y, h: current_state,
                                 learning_rate: LEARNING_RATE, p_keep: DROPOUT_P_KEEP}

                    summary, _, next_state, c, accuracy_ = sess.run([merged, optimizer, states, cost, accuracy],
                                                                    feed_dict=feed_dict)

                    epoch_loss = c
                    current_state = next_state

                if epoch % 2 == 0:
                    saver.save(sess, CHECKPOINT_PATH + MODEL_NAME, global_step=epoch)
                writer.add_summary(summary, epoch)
                print('Epoch : {} completed out of {}, loss : {}, accuracy : {}'.format(epoch, EPOCHS,
                                                                                        epoch_loss, accuracy_))
        except KeyboardInterrupt:
            print('Training interrupted at {}'.format(epoch))
        finally:
            writer.close()

        saver.save(sess, CHECKPOINT_PATH + MODEL_NAME, global_step=epoch)

        x_ = data.test.images.reshape((-1, NUM_CHUNKS, CHUNK_SIZE))
        y_ = data.test.labels
        y_[y_ == 0] = -1

        accuracy_ = sess.run(accuracy, feed_dict={x: x_, y: y_,
                                                  h: np.zeros([10000, CELL_SIZE]),
                                                  p_keep: 1.0})

        print('Accuracy : {}'.format(accuracy_))


if __name__ == '__main__':
    train_neural_network(x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/model.ckpt-2
Epoch : 0 completed out of 100, loss : 18.976234436035156, accuracy : 0.98828125
Epoch : 1 completed out of 100, loss : 26.398448944091797, accuracy : 0.984375
Epoch : 2 completed out of 100, loss : 9.485129356384277, accuracy : 0.99609375
Epoch : 3 completed out of 100, loss : 24.194019317626953, accuracy : 0.9921875
Epoch : 4 completed out of 100, loss : 16.69883155822754, accuracy : 0.98828125
Epoch : 5 completed out of 100, loss : 20.391862869262695, accuracy : 0.9921875
Epoch : 6 completed out of 100, loss : 15.472428321838379, accuracy : 0.9921875
Epoch : 7 completed out of 100, loss : 6.145691871643066, accuracy : 0.99609375
Epoch : 8 completed out of 100, loss : 17.344924926757812, accuracy : 0.984375
Epoch : 9 completed out of 100, loss : 22.249752044677734, accuracy : 0.98046875
Epoch : 10 completed out of 100, loss : 16.3696